<a id="cabecera"></a> 
<img src="images/UCLM_todo_Color.png" alt="Logo UCLM" align="right" width="30%" height="30%">
<br><br><br>




<h2><font color="#004D7F" size=4>Quantum Key Distribution</font></h2>


<h1><font color="#004D7F" size=5>Protocolo BB84</font></h1>
<br><br>

<div style="text-align: right">
<font color="#004D7F" size=3>Hernán Indíbil de la Cruz Calvo</font><br>
<font color="#004D7F" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#004D7F" size=3>Universidad de Castilla-La Mancha</font>

</div>

---
<a id="indice"></a>
<h1><font color="#004D7F" size=5>Índice de Contenido</font></h1>

0. [Preparación del entorno](#section0)
1. [Intercambio de claves](#section1)
    1. [Generación de la clave](#section11)
    2. [Envío](#section12)
    3. [Recepción](#section13)
    4. [Comparación de b y b'](#section14)
2. [Comprobación](#section2)
---
<a id="section0"></a>
# <font color="#004D7F">0. Preparación del entorno</font>

In [1]:
import numpy as np
import random as rnd
import qsimov as qj

---
<a id="section1"></a>
# <font color="#004D7F">1. Intercambio de claves</font>
Vamos a considerar tres actores y un canal en el que puede existir ruido.
Los tres actores son:
* Alice: Inicia el protocolo para intercambiar una clave con Bob
* Bob: Participa en el intercambio de claves con Alice
* Eve: Agente que tratará por todos los medios posibles conseguir la clave

In [2]:
# Determinamos el número de bits a utilizar
n = 2048
noise_prob = 0.05
seed = 6470
rnd.seed(seed)
np.random.seed(seed)
qj.setRandomSeed(seed)

<a id="section11"></a>
## <font color="#004D7F">1.1. Generación de la clave</font>
Alice comienza generando dos tiras de bits de forma aleatoria llamadas <i>a</i> y <i>b</i>.

In [3]:
a = np.array([rnd.randint(0, 1) for i in range(n)])
b = np.array([rnd.randint(0, 1) for i in range(n)])

In [4]:
a

array([1, 1, 0, ..., 0, 1, 1])

In [5]:
b

array([0, 1, 1, ..., 1, 1, 1])

Ahora se procede a crear la lista de QuBits que Alice transmitirá a Bob. El valor de <i>a</i> indicará el valor inicial de cada QuBit a enviar, y el valor de <i>b</i> indica la base en la que se codificará: 0 para base computacional, 1 para base Hadamard. Para cambiar a la base de Hadamard sólo es necesario aplicar la puerta Hadamard.

In [6]:
q = qj.QSystem(n * 2)
for i in range(n):
    if a[i] == 1:
        q.applyGate("X", qubit=i)
    if b[i] == 1:
        q.applyGate("H", qubit=i)

<a id="section12"></a>
## <font color="#004D7F">1.2. Envío</font>
Alice procede a enviar por un canal cuántico los QuBits. En este apartado se simulará el ruido y será donde Eve realice su ataque.
Comenzamos provocando el ruido aplicando una puerta z con una cierta probabilidad.

In [7]:
for i in range(n):
    if rnd.random() < noise_prob:
        q.applyGate("Rz(" + str(np.pi/8) + ")", qubit=i)

Ahora Eve procedería a hacer su jugada. Como el principio de indeterminación hace que sus intentos de descubrir algo de los QuBits afecten a los mismos, sólo le quedaría clonar los QuBits transmitidos. Dado que el teorema de no clonación lo impide, lo único que puede hacer es usar C-NOT, de forma que sus QuBit sean una copia de los que estaban en base computacional y queden entrelazados con los que no.

In [8]:
for i in range(n):
    q.applyGate("X", qubit=i+n, control=i)

<a id="section13"></a>
## <font color="#004D7F">1.3. Recepción</font>
Bob recibe los QuBits enviados por Alice a través del canal inseguro. Comienza generando una tira de bits <i>b'</i> que determina la base en la que tratará de medir. Bob tiene un 50% de probabilidades de acertar para un cierto QuBit <i>q<sub>i</sub></i> se cumpla que <i>b<sub>i</sub></i> = <i>b'<sub>i</sub></i>.

In [9]:
b_prime = np.array([rnd.randint(0, 1) for i in range(n)])

for i in range(n):
    if b_prime[i] == 1:
        q.applyGate("H", qubit=i)

Una vez generado <i>b'</i> y cambiada la base se procede a la medición. En los registros consideramos que el primer QuBit es el original, mientras que el segundo es el que tiene Eve.

In [10]:
a_prime = np.array(q.measure([1 if i < n else 0 for i in range(n * 2)], remove=True))
print(a_prime)

[0 1 0 ... 1 0 0]


<a id="section14"></a>
## <font color="#004D7F">1.4. Comparación de <i>b</i> y <i>b'</i></font>
Bob procede a anunciar que ha terminado de realizar las mediciones, momento en el que Alice publica <i>b</i>. Bob entonces procede a compararlo con <i>b'</i> para saber qué valores se han medido correctamente. Si no se ha producido ruido o un intento de escucha, <i>b<sub>i</sub></i> = <i>b'<sub>i</sub></i> &rArr; <i>a<sub>i</sub></i> = <i>a'<sub>i</sub></i>.

In [11]:
aciertos = b == b_prime
aciertos

array([False, False, False, ...,  True, False, False])

Y finalmente Bob comunica la lista de aciertos. Todos los QuBits para los que no se haya producido un acierto serán descartados tanto por Alice como por Bob (y potencialmente por Eve ya que no formarán parte de la clave).

In [12]:
shared_a = np.array([a[i] for i in range(n) if aciertos[i]])
shared_a_prime = np.array([a_prime[i] for i in range(n) if aciertos[i]])

Además, Eve ahora sabe que todos los que tienen acierto y originalmente fueron codificados en base computacional fueron copiados correctamente. En cuanto a los que tienen acierto y estaban en base hadamard, bastará con aplicarles una Hadamard y medir.

In [13]:
for i in range(n):
    if aciertos[i] and b[i] == 1:
        q.applyGate("H", qubit=i+n)

shared_a_eve = np.array(q.measure([0 if i < n or not aciertos[i - n] else 1 for i in range(n * 2)]))

Vamos a comprobar si efectivamente Eve tiene la clave

In [14]:
all(shared_a_prime == shared_a_eve)

False

Se puede observar que no. Vamos a ver en qué casos ha fallado.

In [15]:
data = np.array([(a[i], b[i], b_prime[i]) for i in range(n) if aciertos[i]])
failed = np.array([data[i] for i in range(len(data)) if not(shared_a_prime == shared_a_eve)[i]])
failed

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1,

Se puede observar que Eve falla en los casos en los que se ha codificado un 1 en base de Hadamard. Veamos qué sucede:

In [16]:
r = qj.QSystem(2)
# Codificación hecha por Alice
r.applyGate("X", "I")
r.applyGate("H", "I")
# Fan-out de Eve
r.applyGate("X", qubit=1, control=0)
# Decodificación de Bob
r.applyGate("H", "I")
r.getState()

array([ 0.5+0.j,  0.5+0.j, -0.5+0.j,  0.5+0.j])

Se puede observar que Bob tiene un 50% de probabilidades de obtener cada uno de los posibles valores al medir, al contrario que en el caso en el que Eve no interfiere, donde tendría un 100% de obtener el correcto. Eve ha alterado el QuBit enviado.

In [17]:
# Lectura de Bob
mes = r.measure([1, 0], remove=True)
print("Medida: " + str(mes[0]))
r.getState()

Medida: 0


array([ 0.70710678+0.j, -0.70710678+0.j])

In [18]:
# Recuperación de Eve
r.applyGate("H")
mes = r.measure([0, 1])
print("Medida: " + str(mes[0]))
r.getState()

Medida: 1


array([0.+0.j, 1.+0.j])

Debido al uso de CNOT por parte de Eve, el QuBit enviado por Alice se ha visto afectado y en algunos casos, aun teniendo la base correcta, la medida da un valor incorrecto a Bob.

La intrusión ha afectado al sistema dando valores aleatorios en algunas medidas correctas de Bob. Concretamente todas las medidas que ha realizado Bob en base de Hadamard le han dado un valor aleatorio (cuando se debería haber recuperado el valor original).

Por ello, teniendo que la mitad (en media) de los valores están en dicha base y (en media) Bob fallará la mitad, un 25% (también en media) de los valores que Bob creía correctos serán erroneos.

Para Eve, debido al estado entrelazado que ha formado con Bob, cuando el número codificado en base de Hadamard era un 0 Eve tendrá siempre el mismo número aleatorio que haya obtenido Bob. Cuando el número codificado era un 1, Eve tendrá siempre el contrario que el medido por Bob.

---
<a id="section2"></a>
# <font color="#004D7F">2. Comprobación</font>
Ahora Alice y Bob comparten la mitad de la clave escogida de forma aleatoria para compararla. Si no concuerdan, puede significar que ha habido ruido o una intrusión.

In [19]:
# Elección de los bits a compartir
randids = [i for i in range(len(shared_a))]
rnd.shuffle(randids)
shared_ids = randids[:len(shared_a)//2]

# Comparación de los bits compartidos
result = np.array([int(shared_a[i] == shared_a_prime[i]) for i in shared_ids])

Finalmente calculamos la tasa de error. Si es mayor que un cierto valor (sobre 11% habitualmente) significa que ha habido demasiado ruido o una intrusión, por lo que se descarta la clave compartida y se vuelve a empezar.

In [20]:
error = 1 - sum(result)/(len(shared_a)//2)
print("Tasa de error: " + str(error * 100) + " %")

Tasa de error: 25.631067961165044 %


Como es mayor, se volvería a comenzar. Si fuese menor se procedería a utilizar métodos clásicos para descartar todos los bits posibles que no concuerden.

---